In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models


In [2]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to match the model input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # Normalize images
])


In [3]:
train_dataset = datasets.ImageFolder(r'C:\Users\satya\Desktop\ransom\cnn2\MORE\RGBprocessed_images\train', transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = datasets.ImageFolder(r'C:\Users\satya\Desktop\ransom\cnn2\MORE\RGBprocessed_images\test', transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)


In [5]:
model = models.resnet50(pretrained=True)  # Example of a BiT-compatible architecture
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_dataset.classes))  # Adjust for your classes


In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(10):  # Number of epochs
    model.train()
    running_loss = 0.0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")


Epoch 1, Loss: 0.11576137686534686
Epoch 2, Loss: 0.04562184171787676
Epoch 3, Loss: 0.055077473599243254
Epoch 4, Loss: 0.10773942921189295
Epoch 5, Loss: 0.04888235486206658
Epoch 6, Loss: 0.06368099596410831
Epoch 7, Loss: 0.036115333820758835
Epoch 8, Loss: 0.03623660149094847
Epoch 9, Loss: 0.03656018546539696
Epoch 10, Loss: 0.07846114911963208


In [8]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")


Test Accuracy: 98.34%
